# Лабораторные работы 5-8
## Нейронные сети
Для начала импортируем модуль **numpy** для удобного хранения и обработки данных

In [1]:
import numpy as np

np.random.seed(1)

Далее опишим некоторые из функций активации (такие как **relu**, **sigmoid**, **tanh** и тп), которые будут использоваться в дальнейшем.

In [2]:
def relu(x, rev=False):
    if not rev:
        return (x > 0) * x
    else:
        return x > 0

    
def sigmoid(x, rev=False):
    if not rev:
        return 1 / (1 + np.exp(-x))
    else:
        return sigmoid(x) * (1 - sigmoid(x))

    
def tanh(x, rev=False):
    if not rev:
        return np.tanh(x)
    else:
        return 1 - (x ** 2)


def liner(x):
    return x


def softmax(x):
    temp = np.exp(x)
    return temp / np.sum(temp, axis=1, keepdims=True)

Теперь напишим основной класс для нейронной сети, который бует использоваться для решения последующих работ.  
Данный класс будет иметь параметры:
* dataset - сами данные на основе которых будет производится обучения
* answer - правильные ответы для входных данных, так же используются для обучения
* neural_count - колличество нейронов в скрытом слое
* iteration_count - колличесвто эпох обучения
* alpha - коэффициент используемый для уменьшения шага весов при обучении
* hidden_layer_activation_func - функция активации для скрытого слоя
* output_layer_activation_func - функция активации для выходного слоя
* weights_0_1 - веса скрытого слоя
* weights_1_2 - веса выходного слоя

А так же методы:
* generate_weights - генерирует случайные веса для данной сети
* train - основной метод обучения нерйронной сети
* predict - делает предсказание используя веса уже обученной сети

In [3]:
class NeuralNetwork:
    def __init__(self, dataset, answer, neural_count=6, iteration_count=10000, alpha=0.2, func1=relu, func2=liner):
        self.dataset = dataset
        self.answer = answer
        self.neural_count = neural_count
        self.iteration_count = iteration_count
        self.alpha = alpha
        self.hidden_layer_activation_func = func1
        self.output_layer_activation_func = func2
        self.weights_0_1 = 0
        self.weights_1_2 = 0
        self.GenerateWeights()

    def GenerateWeights(self):
        self.weights_0_1 = 2*np.random.sample((len(self.dataset[0]), self.neural_count)) - 1
        self.weights_1_2 = 2*np.random.sample((self.neural_count, len(self.answer[0]))) - 1
        
    def Train(self, dropout=False):
        from time import time
        start = time()
        for iteration in range(self.iteration_count):
            layer_2_error = 0
            for i in range(len(self.dataset)):
                self.layer_0 = self.dataset[i:i+1]
                self.Predict(self.layer_0)
                layer_2_error += np.sum((self.layer_2 - self.answer[i:i+1]) ** 2)
                
                if sigmoid in (self.hidden_layer_activation_func, self.output_layer_activation_func):
                    self.ce(i)
                else:
                    self.mse(i)

            if iteration % (self.iteration_count // 10) == 0:
                print('На итерации', iteration, 'ошибка -', layer_2_error)

        print('_' * 50)
        print('Итоговая ошибка', layer_2_error)
        print()
        print('Обучение заняло', time()-start, 'сек')

    def mse(self, i):
        layer_2_delta = self.layer_2 - self.answer[i:i+1]
        layer_1_delta = layer_2_delta.dot(self.weights_1_2.T) \
                                * self.hidden_layer_activation_func(self.layer_1, rev=True)
        
        self.weights_1_2 -= self.alpha * self.layer_1.T.dot(layer_2_delta)
        self.weights_0_1 -= self.alpha * self.layer_0.T.dot(layer_1_delta)
        
    def ce(self, i):
        layer_2_delta = np.dot(
            self.layer_1.T,
            (2 * (self.answer[i:i+1] - self.layer_2) * self.output_layer_activation_func(self.layer_2, rev=True))
        )

        layer_1_delta = np.dot(
            self.layer_0.T,
            np.dot(
                2*(self.answer[i:i+1] - self.layer_2) * self.output_layer_activation_func(self.layer_2, rev=True),
                self.weights_1_2.T) * self.hidden_layer_activation_func(self.layer_1, rev=True)
        )

        self.weights_1_2 += self.alpha * layer_2_delta
        self.weights_0_1 += self.alpha * layer_1_delta

    def Predict(self, data, dropout=False):
        self.layer_1 = self.hidden_layer_activation_func(np.dot(data, self.weights_0_1))
        self.layer_2 = self.output_layer_activation_func(np.dot(self.layer_1, self.weights_1_2))

        if not (data is self.layer_0):
            return self.layer_2

# Лабораторная работа 5
## Создание и обучение простейшей нейронной сети

**Цель** - создание и обучение простейшей нейронной сети для решения задачи XOR для двух элементов.  
Для начала в переменные ***xor*** и ***xor_answer*** записываем данные на которых будет производится обучение модели.

In [4]:
xor = np.array([[0,0],
                [0,1],
                [1,0],
                [1,1]])
xor_answer = np.array([[0,1,1,0]]).T

Далее создаем объект Нейронной сети для данной задачи. Будем использовать 4 нейрона в скрытом слое и 1000 эпох обучения.

In [5]:
Xor = NeuralNetwork(dataset=xor,
                    answer=xor_answer,
                    alpha=0.1,
                    neural_count=4,
                    iteration_count=500)
Xor.GenerateWeights()

После случайной генерации веса имеют значения:

In [6]:
Xor.weights_0_1

array([[-0.5910955 ,  0.75623487, -0.94522481,  0.34093502],
       [-0.1653904 ,  0.11737966, -0.71922612, -0.60379702]])

In [7]:
Xor.weights_1_2

array([[ 0.60148914],
       [ 0.93652315],
       [-0.37315164],
       [ 0.38464523]])

Произведем обучение модели.

In [8]:
Xor.Train()

На итерации 0 ошибка - 1.6825749810337902
На итерации 50 ошибка - 0.27516577442941104
На итерации 100 ошибка - 2.1357415879146058e-05
На итерации 150 ошибка - 1.8484091669274243e-10
На итерации 200 ошибка - 1.5791204071587323e-15
На итерации 250 ошибка - 1.3490112339976078e-20
На итерации 300 ошибка - 1.1525550683090072e-25
На итерации 350 ошибка - 1.8504634031807123e-30
На итерации 400 ошибка - 4.6995681904394146e-31
На итерации 450 ошибка - 4.6995681904394146e-31
__________________________________________________
Итоговая ошибка 4.6995681904394146e-31

Обучение заняло 0.07262253761291504 сек


Теперь веса в нашей модели имеют значения:

In [9]:
Xor.weights_0_1

array([[-0.5910955 , -0.82800213, -0.94522481,  0.89473936],
       [-0.1653904 ,  0.82800213, -0.71922612, -0.90029912]])

In [10]:
Xor.weights_1_2

array([[ 0.60148914],
       [ 1.20772637],
       [-0.37315164],
       [ 1.11764391]])

Протестируем работу обученной модели, посмотрим какой результат она выдаст на наши входные данные.

In [11]:
print('По итогу обучения модель считает:')
for i in range(len(xor)):
    data = xor[i:i+1]
    print(f'{data[0][0]} XOR {data[0][1]} = {round(Xor.Predict(data)[0][0])}')

По итогу обучения модель считает:
0 XOR 0 = 0
0 XOR 1 = 1
1 XOR 0 = 1
1 XOR 1 = 0


***Вывод***  
В данной работе нам удалось построить свою первую нейронную сеть состояющую из 4 нейроннов скрытого слоя и одного выходного, которая решает задачу *XOR* между 0 и 1. По итогам теста данная сеть на 100 справилась со своей задачей, а итоговая ошибка составила всего лишь 4.6995681904394146e-31. 

# Лабораторная работа 6
## Определение напрвления двоичного сдвига

**Цель** - построение, обучение и тестирование нейронной сети, предназначенной для определения направления двоичного кода.  
Первое что необходимо сделать, это сгенерировать входные данные, которые будут представлять из себя матрицу. В данной матрице строка, это изначльные нули и еденицы, в колличестве равном значению *count* (5 штук), а так же их представление уже со сдвигом.


In [12]:
count = 5

binary = [list(bin(i)[2:].zfill(5)) for i in range(1, 2**count-1)]
binary = [i + i[1:] + i[0:1] for i in binary] + [i + i[-1:] + i[:len(i)-1] for i in binary]
binary = np.array(binary, dtype=np.int)
binary_answer = np.array([[*[0 for i in range(2**count-2)], *[1 for i in range(2**count-2)] ]]).T 

binary

array([[0, 0, 0, 0, 1, 0, 0, 0, 1, 0],
       [0, 0, 0, 1, 0, 0, 0, 1, 0, 0],
       [0, 0, 0, 1, 1, 0, 0, 1, 1, 0],
       [0, 0, 1, 0, 0, 0, 1, 0, 0, 0],
       [0, 0, 1, 0, 1, 0, 1, 0, 1, 0],
       [0, 0, 1, 1, 0, 0, 1, 1, 0, 0],
       [0, 0, 1, 1, 1, 0, 1, 1, 1, 0],
       [0, 1, 0, 0, 0, 1, 0, 0, 0, 0],
       [0, 1, 0, 0, 1, 1, 0, 0, 1, 0],
       [0, 1, 0, 1, 0, 1, 0, 1, 0, 0],
       [0, 1, 0, 1, 1, 1, 0, 1, 1, 0],
       [0, 1, 1, 0, 0, 1, 1, 0, 0, 0],
       [0, 1, 1, 0, 1, 1, 1, 0, 1, 0],
       [0, 1, 1, 1, 0, 1, 1, 1, 0, 0],
       [0, 1, 1, 1, 1, 1, 1, 1, 1, 0],
       [1, 0, 0, 0, 0, 0, 0, 0, 0, 1],
       [1, 0, 0, 0, 1, 0, 0, 0, 1, 1],
       [1, 0, 0, 1, 0, 0, 0, 1, 0, 1],
       [1, 0, 0, 1, 1, 0, 0, 1, 1, 1],
       [1, 0, 1, 0, 0, 0, 1, 0, 0, 1],
       [1, 0, 1, 0, 1, 0, 1, 0, 1, 1],
       [1, 0, 1, 1, 0, 0, 1, 1, 0, 1],
       [1, 0, 1, 1, 1, 0, 1, 1, 1, 1],
       [1, 1, 0, 0, 0, 1, 0, 0, 0, 1],
       [1, 1, 0, 0, 1, 1, 0, 0, 1, 1],
       [1, 1, 0, 1, 0, 1,

Увеличим колличество нейроннов в скрытом слое по сравнению с первой работой до 10, так как здесь поиск закономерности является более сложной задачей.

In [13]:
Binary = NeuralNetwork(dataset=binary,
                       answer=binary_answer,
                       neural_count=10)

Произведем обучение модели.

In [14]:
Binary.Train()

На итерации 0 ошибка - 18.97740014099182
На итерации 1000 ошибка - 0.5684430473562058
На итерации 2000 ошибка - 0.3275776557933158
На итерации 3000 ошибка - 0.22605799259031428
На итерации 4000 ошибка - 0.33339584387096827
На итерации 5000 ошибка - 0.00013536439910043454
На итерации 6000 ошибка - 1.3588924830368137e-06
На итерации 7000 ошибка - 1.5254175183160808e-08
На итерации 8000 ошибка - 1.496170081031347e-08
На итерации 9000 ошибка - 1.3966431377696958e-09
__________________________________________________
Итоговая ошибка 1.6991406033880625e-09

Обучение заняло 17.83565378189087 сек


Протестируем работу обученной модели. Посмотрим какой результат она выдаст на наши входные данные.

In [15]:
from random import randint


test = np.array([binary[randint(0, 59)]for i in range(10)])
for i in range(len(test)):
    data = test[i:i+1]
    ans =  'вправо' if round(Binary.Predict(data)[0][0]) else 'влево'
    print(f'{data[0][:count]} -> {data[0][count:]} - сдвиг {ans}')


[0 1 0 0 1] -> [1 0 0 1 0] - сдвиг влево
[1 1 0 0 1] -> [1 1 1 0 0] - сдвиг вправо
[0 1 1 0 1] -> [1 1 0 1 0] - сдвиг влево
[1 1 0 0 1] -> [1 0 0 1 1] - сдвиг влево
[0 0 1 1 0] -> [0 0 0 1 1] - сдвиг вправо
[1 1 0 0 1] -> [1 0 0 1 1] - сдвиг влево
[1 0 0 0 1] -> [1 1 0 0 0] - сдвиг вправо
[1 0 1 1 1] -> [1 1 0 1 1] - сдвиг вправо
[1 0 1 0 0] -> [0 1 0 1 0] - сдвиг вправо
[0 0 1 0 1] -> [0 1 0 1 0] - сдвиг влево


**Вывод**  
Данная нейросеть ищет более сложные закономероности, поэтому нуждается в большем колличестве нейронов. По итогом произведенного обучения итоговая ошибка составила 1.6991406033880625e-09, что является хорошим результатом. Значит после произведенного обучения нейронная сети отлично справляется с поставленной задачей. Это хорошо заметно на результатх теста.

# Лабораторная работа 7.1.
## Распознавание символов

**Цель** - разработать и исслодовать нейронную сеть обратного распределения предназначенную для распознавания образов.  
В переменные ***lyters*** и ***lyters_answer*** записываем данные на которых будет производится обучение модели.

In [16]:
lyters = np.array([[1,0,1,0,1,0,1,0,1],
                   [1,0,1,0,1,0,0,1,0],
                   [0,1,0,0,1,0,0,1,0],
                   [1,1,1,1,0,0,1,1,1]])

lyters_answer = np.array([[0, 0, 0, 1],  # X
                          [0, 0, 1, 0],  # Y
                          [0, 1, 0, 0],  # I
                          [1, 0, 0, 0]]) # C

Создадим объект нейронной сети. В скрытом слою будем использовать 15 нейронов, а так же возьмем колличество эпох равное 20000. До этого случая у нас был один выход *(1 или 0)*, но сейчас нам нужно классифицировать 4 разные буквы, и выходов для этого мы будем использовать 4 соответвенно, поэтому здесь в качестве функции активации выходного слоя лучше использовать функцию softmax.

In [17]:
Lyters = NeuralNetwork(dataset=lyters,
                       answer=lyters_answer,
                       neural_count=8,
                       func2=softmax)

Произведем обучение модели.

In [18]:
Lyters.Train()

На итерации 0 ошибка - 4.6307497000722755
На итерации 1000 ошибка - 3.598782996852221e-07
На итерации 2000 ошибка - 7.395670893700428e-08
На итерации 3000 ошибка - 2.9716082815849802e-08
На итерации 4000 ошибка - 1.560325762407876e-08
На итерации 5000 ошибка - 9.479222995689099e-09
На итерации 6000 ошибка - 6.313220817519294e-09
На итерации 7000 ошибка - 4.479650956636285e-09
На итерации 8000 ошибка - 3.3292990403212777e-09
На итерации 9000 ошибка - 2.5633582396200277e-09
__________________________________________________
Итоговая ошибка 2.029559646661728e-09

Обучение заняло 2.276326894760132 сек


Протестируем работу обученной модели. Посмотрим какой результат она выдаст на наши входные данные.

In [19]:
print('По итогу обучения модель считает:')
for i in range(len(lyters)):
    data = lyters[i:i+1]
    ans = list(map(round, Lyters.Predict(data)[0]))
    print(f'{data[0]} - {ans}')

По итогу обучения модель считает:
[1 0 1 0 1 0 1 0 1] - [0, 0, 0, 1]
[1 0 1 0 1 0 0 1 0] - [0, 0, 1, 0]
[0 1 0 0 1 0 0 1 0] - [0, 1, 0, 0]
[1 1 1 1 0 0 1 1 1] - [1, 0, 0, 0]


**Вывод**  
Так как сумма всех значений выходного слоя после использования **softmax** равна 1, несложно произвести классификацию при помощи округления значений. Итоговая точность значения будет 2.029559646661728e-09 соответсвенно.

## Дополнительные задания к практической работе No 7.
### Задание 7.1.1.
**Задание:** cоздать нейронную сеть со структурой «многослойный персептрон», и
обучить ее распознаванию цифр, заданных пиксельной матрицей размером
7х5, используя 4 битный код на выходе. Произвести тестирование НС при
добавлении шума.

Изначально цифры будут выглядеть так:
![image0](https://raw.githubusercontent.com/sergo2048/Data-mining/main/Neural%20networks/images/0.png)
![image1](https://raw.githubusercontent.com/sergo2048/Data-mining/main/Neural%20networks/images/1.png)
![image2](https://raw.githubusercontent.com/sergo2048/Data-mining/main/Neural%20networks/images/2.png)
![image3](https://raw.githubusercontent.com/sergo2048/Data-mining/main/Neural%20networks/images/3.png)
![image4](https://raw.githubusercontent.com/sergo2048/Data-mining/main/Neural%20networks/images/4.png)
![image5](https://raw.githubusercontent.com/sergo2048/Data-mining/main/Neural%20networks/images/5.png)
![image6](https://raw.githubusercontent.com/sergo2048/Data-mining/main/Neural%20networks/images/6.png)
![image7](https://raw.githubusercontent.com/sergo2048/Data-mining/main/Neural%20networks/images/7.png)
![image8](https://raw.githubusercontent.com/sergo2048/Data-mining/main/Neural%20networks/images/8.png)
![image9](https://raw.githubusercontent.com/sergo2048/Data-mining/main/Neural%20networks/images/9.png)

При переносе в матричный вид за еденицу будем считать черный цвет, за 0 белый.

In [20]:
numbers = np.array([[0,0,1,0,0,
                     0,0,1,0,0,
                     0,0,1,0,0,
                     0,0,1,0,0,
                     0,0,1,0,0,
                     0,0,1,0,0,
                     0,0,1,0,0],
  
                    [1,1,1,1,1,
                     0,0,0,0,1,
                     0,0,0,0,1,
                     1,1,1,1,1,
                     1,0,0,0,0,
                     1,0,0,0,0,
                     1,1,1,1,1],

                    [1,1,1,1,1,
                     0,0,0,0,1,
                     0,0,0,0,1,
                     1,1,1,1,1,
                     0,0,0,0,1,
                     0,0,0,0,1,
                     1,1,1,1,1],

                    [1,0,0,0,1,
                     1,0,0,0,1,
                     1,0,0,0,1,
                     1,1,1,1,1,
                     0,0,0,0,1,
                     0,0,0,0,1,
                     0,0,0,0,1],

                    [1,1,1,1,1,
                     1,0,0,0,0,
                     1,0,0,0,0,
                     1,1,1,1,1,
                     0,0,0,0,1,
                     0,0,0,0,1,
                     1,1,1,1,1],
                    
                    [1,1,1,1,1,
                     1,0,0,0,0,
                     1,0,0,0,0,
                     1,1,1,1,1,
                     1,0,0,0,1,
                     1,0,0,0,1,
                     1,1,1,1,1],
  
                    [1,1,1,1,1,
                     0,0,0,0,1,
                     0,0,0,0,1,
                     0,0,0,0,1,
                     0,0,0,0,1,
                     0,0,0,0,1,
                     0,0,0,0,1],

                    [1,1,1,1,1,
                     1,0,0,0,1,
                     1,0,0,0,1,
                     1,1,1,1,1,
                     1,0,0,0,1,
                     1,0,0,0,1,
                     1,1,1,1,1],

                    [1,1,1,1,1,
                     1,0,0,0,1,
                     1,0,0,0,1,
                     1,1,1,1,1,
                     0,0,0,0,1,
                     0,0,0,0,1,
                     1,1,1,1,1],

                    [1,1,1,1,1,
                     1,0,0,0,1,
                     1,0,0,0,1,
                     1,0,0,0,1,
                     1,0,0,0,1,
                     1,0,0,0,1,
                     1,1,1,1,1]], dtype=np.float128)


numbers_answer = np.array([[0, 0, 0, 1],  # 1
                           [0, 0, 1, 0],  # 2
                           [0, 0, 1, 1],  # 3
                           [0, 1, 0, 0],  # 4
                           [0, 1, 0, 1],  # 5
                           [0, 1, 1, 0],  # 6
                           [0, 1, 1, 1],  # 7
                           [1, 0, 0, 0],  # 8
                           [1, 0, 0, 1],  # 9
                           [1, 0, 1, 0]]) # 0

Данная нейронная сеть должны быть умнее, поэтому здесь нам не будет хватать relu функции. Поэтому в дальнейшем мы будем использовать sigmoid в качестве функции активации, а так же кросс-энтропию в качестве функции потерь.

In [21]:
Numbers = NeuralNetwork(dataset=numbers,
                        answer=numbers_answer,
                        neural_count=15,
                        iteration_count=20000,
                        func1=sigmoid,
                        func2=sigmoid)

Произведем обучение модели, путем изменения весовых коэффициентов.

In [22]:
Numbers.Train()

На итерации 0 ошибка - 10.502539468463692553
На итерации 2000 ошибка - 0.024845935606401343265
На итерации 4000 ошибка - 0.004947737261356947899
На итерации 6000 ошибка - 0.0015568962294123111726
На итерации 8000 ошибка - 0.0007166011025013609525
На итерации 10000 ошибка - 0.00040214141985363674558
На итерации 12000 ошибка - 0.0002494918778085184155
На итерации 14000 ошибка - 0.00016518246638597386531
На итерации 16000 ошибка - 0.00011550992393597822489
На итерации 18000 ошибка - 8.678389071532254124e-05
__________________________________________________
Итоговая ошибка 7.190813318555330623e-05

Обучение заняло 18.669897317886353 сек


Теперь добавим некий шум к нашим исходным данным.

In [23]:
numbers_mod = np.array([[0,0,1,0,0,
                         0,1,1,0,0,
                         0,0,1,0,0,
                         0,0,1,0,0,
                         0,0,1,0,0,
                         0,0,1,0,0,
                         0,0,1,0,0],

                        [1,1,1,1,0,
                         0,0,0,0,1,
                         0,0,0,0,1,
                         1,1,1,1,1,
                         1,0,0,0,0,
                         1,0,0,0,0,
                         0,1,1,1,1],

                        [1,1,1,1,0,
                         0,0,0,0,1,
                         0,0,0,0,1,
                         0,1,1,1,1,
                         0,0,0,0,1,
                         0,0,0,0,1,
                         1,1,1,1,1],

                        [1,0,0,0,1,
                         1,0,0,0,1,
                         1,0,0,0,1,
                         0,1,1,1,1,
                         0,0,0,1,1,
                         0,0,0,0,1,
                         0,0,0,0,1],

                        [0,1,1,1,1,
                         1,0,0,0,0,
                         1,0,0,0,0,
                         1,1,1,1,1,
                         0,0,0,0,1,
                         0,0,0,0,1,
                         1,1,1,1,1],

                        [1,1,1,1,1,
                         1,0,0,0,1,
                         1,0,0,0,0,
                         1,1,1,1,0,
                         1,0,0,0,1,
                         1,0,0,0,1,
                         1,1,1,1,1],

                        [1,1,1,1,1,
                         0,0,0,0,1,
                         0,0,0,0,1,
                         0,0,0,1,1,
                         0,0,0,0,1,
                         0,0,0,0,1,
                         0,0,0,0,1],

                        [0,1,1,1,0,
                         1,0,0,0,1,
                         1,0,0,1,1,
                         1,1,1,1,1,
                         1,0,0,0,1,
                         1,0,0,0,1,
                         1,1,1,1,1],

                        [1,1,1,1,1,
                         1,0,0,0,1,
                         1,0,0,0,1,
                         0,1,1,1,1,
                         0,0,0,0,1,
                         0,0,0,0,1,
                         1,1,1,1,1],

                        [0,1,1,1,0,
                         1,0,0,0,1,
                         1,0,0,0,1,
                         1,0,0,0,1,
                         1,0,0,0,1,
                         1,0,0,0,1,
                         0,1,1,1,0]], dtype=np.float128)


Далее протестируем работу обученной модели, на модифицированных данных и посмотрим какой результат она выдаст.

In [24]:
print('На модифицированные данные нейронная сеть отвечает:')
error = 0

for i in range(len(numbers_mod)):
    data = numbers_mod[i:i+1]
    ans = Numbers.Predict(data)
    error += np.sum((ans - Numbers.answer[i:i+1]) ** 2)
    print(f'{ans[0]}')

print(f'Ошибка в предсказании составила - {error}')

На модифицированные данные нейронная сеть отвечает:
[1.91543453e-08 6.10578145e-04 1.12144584e-04 9.99960418e-01]
[9.84597373e-01 1.30963527e-08 2.50615712e-01 4.08721141e-06]
[8.97895358e-01 2.14842656e-04 1.12384944e-02 9.99971155e-01]
[6.08719616e-06 9.99993867e-01 6.20477327e-14 7.23432953e-07]
[0.13311161 0.89523078 0.08161367 0.65970902]
[0.5 0.5 0.5 0.5]
[0.30234073 0.57035338 0.556819   0.72483873]
[9.99999955e-01 2.55151182e-08 2.38825608e-08 3.76157350e-04]
[9.98108071e-01 1.23138037e-03 8.20926681e-04 9.98081711e-01]
[9.99999955e-01 2.55423262e-08 2.39095719e-08 3.75783349e-04]
Ошибка в предсказании составила - 6.014167595076182


### Задание 7.1.3.
**Задание:** Создать нейронную сеть со структурой «многослойный персептрон», и обучить ее распознаванию четности цифр, заданных пиксельной матрицей размером 7х5. Произвести тестирование НС при добавлении шума.  
В переменную **numbers_odd_answer** записываем данные на которых будет производится обучение модели, а также берем numbers из прошлого задания.

In [25]:
numbers_odd_answer = np.array([[0,1,0,1,0,1,0,1,0,1]]).T

Создадим объект класса NeuralNetwork.

In [26]:
Numbers_odd = NeuralNetwork(dataset=numbers,
                            answer=numbers_odd_answer,
                            neural_count=15,
                            iteration_count=10000,
                            func1=sigmoid,
                            func2=sigmoid)

Произведем обучение.

In [27]:
Numbers_odd.Train()

На итерации 0 ошибка - 3.1027190134881182674
На итерации 1000 ошибка - 0.00016549573241578388573
На итерации 2000 ошибка - 4.1678749263587366035e-05
На итерации 3000 ошибка - 1.8998197187782271325e-05
На итерации 4000 ошибка - 1.0728690188161379414e-05
На итерации 5000 ошибка - 6.7779811369194759064e-06
На итерации 6000 ошибка - 4.6082142750719164525e-06
На итерации 7000 ошибка - 3.3078082907775639422e-06
На итерации 8000 ошибка - 2.477602633940519356e-06
На итерации 9000 ошибка - 1.9203997164728888466e-06
__________________________________________________
Итоговая ошибка 1.5309250264539209353e-06

Обучение заняло 9.536542654037476 сек


Далее протестируем работу обученной модели, на модифицированных данных и посмотрим какой результат она выдаст.

In [28]:
print('По итогу обучения модель считает:')
error = 0

for i in range(len(numbers_mod)):
    data = numbers_mod[i:i+1]
    ans = Numbers_odd.Predict(data)[0][0]
    error += np.sum((ans - Numbers_odd.answer[i:i+1]) ** 2)
    ans = 'Четное' if round(ans) else 'Не четное'
    print(f'{(i + 1) % 10} - {ans}')

print(f'Ошибка в предсказании составила - {error}')

По итогу обучения модель считает:
1 - Не четное
2 - Четное
3 - Не четное
4 - Четное
5 - Не четное
6 - Четное
7 - Не четное
8 - Четное
9 - Не четное
0 - Четное
Ошибка в предсказании составила - 7.898141786469726e-06


### Задание 7.1.4.
**Задание:** Создать нейронную сеть со структурой «многослойный персептрон», и обучить ее распознаванию нечетности цифр, заданных пиксельной матрицей размером 7х5. Произвести тестирование НС при добавлении шума.  
В этого в переменную ***numbers_even_answer*** записываем данные на которых будет производится обучение модели, а также берем numbers из прошлого задания.

In [29]:
numbers_even_answer = np.array([[1,0,1,0,1,0,1,0,1,0]]).T

Создадим объект класса NeuralNetwork.

In [30]:
Numbers_even = NeuralNetwork(dataset=numbers,
                            answer=numbers_even_answer,
                            neural_count=15,
                            iteration_count=10000,
                            func1=sigmoid,
                            func2=sigmoid)

Произведем обучение.

In [31]:
Numbers_even.Train()

На итерации 0 ошибка - 2.6470274485642733447
На итерации 1000 ошибка - 7.941719813097072918e-05
На итерации 2000 ошибка - 2.7339992701597333521e-05
На итерации 3000 ошибка - 1.4785253034511263706e-05
На итерации 4000 ошибка - 9.246420057771774955e-06
На итерации 5000 ошибка - 6.2224355519073858853e-06
На итерации 6000 ошибка - 4.39862226433083125e-06
На итерации 7000 ошибка - 3.2311548305648417518e-06
На итерации 8000 ошибка - 2.45075695545051909e-06
На итерации 9000 ошибка - 1.9103662798935275242e-06
__________________________________________________
Итоговая ошибка 1.5249789766015379178e-06

Обучение заняло 9.544184446334839 сек


Далее протестируем работу обученной модели, на модифицированных данных и посмотрим какой результат она выдаст.

In [32]:
print('По итогу обучения модель считает:')
error = 0

for i in range(len(numbers_mod)):
    data = numbers_mod[i:i+1]
    ans = Numbers_even.Predict(data)[0][0]
    error += np.sum((ans - Numbers_even.answer[i:i+1]) ** 2)
    ans = 'Не четное' if round(ans) else 'Четное'
    print(f'{(i + 1) % 10} - {ans}')

print(f'Ошибка в предсказании составила - {error}')

По итогу обучения модель считает:
1 - Не четное
2 - Четное
3 - Не четное
4 - Четное
5 - Не четное
6 - Четное
7 - Не четное
8 - Четное
9 - Не четное
0 - Четное
Ошибка в предсказании составила - 1.2265074419934752e-05


### Задание 7.1.5.
**Задание:** Создать нейронную сеть со структурой «многослойный персептрон», и обучить ее распознаванию простых чисел (от 0 до 9), заданных пиксельной матрицей размером 7х5. Произвести тестирование НС при добавлении шума.  
Как и до этого в переменную ***numbers_simpe_answer*** записываем данные на которых будет производится обучение модели, а также берем numbers из прошлого задания.

In [33]:
numbers_simple_answer = np.array([[1,1,1,0,1,0,1,0,0,0]]).T

Создадим объект класса NeuralNetwork.

In [34]:
Numbers_simple = NeuralNetwork(dataset=numbers,
                               answer=numbers_simple_answer,
                               neural_count=15,
                               iteration_count=10000,
                               func1=sigmoid,
                               func2=sigmoid)

Произведем обучение.

In [35]:
Numbers_simple.Train()

На итерации 0 ошибка - 3.480763845949079316
На итерации 1000 ошибка - 0.0030942048547330868886
На итерации 2000 ошибка - 0.00078816852956844107963
На итерации 3000 ошибка - 0.00034912105051482272676
На итерации 4000 ошибка - 0.00019486493288574337441
На итерации 5000 ошибка - 0.0001235686368483254216
На итерации 6000 ошибка - 8.498118368715049442e-05
На итерации 7000 ошибка - 6.182688818798110338e-05
На итерации 8000 ошибка - 4.6882165786939831442e-05
На итерации 9000 ошибка - 3.6697137679939213896e-05
__________________________________________________
Итоговая ошибка 2.946307512795766544e-05

Обучение заняло 9.505537748336792 сек


Далее протестируем работу обученной модели, на модифицированных данных и посмотрим какой результат она выдаст.

In [36]:
print('По итогу обучения модель считает:')
error = 0

for i in range(len(numbers_mod)):
    data = numbers_mod[i:i+1]
    ans = Numbers_simple.Predict(data)[0][0]
    error += np.sum((ans - Numbers_simple.answer[i:i+1]) ** 2)
    ans = 'Простое' if round(ans) else 'Не простое'
    print(f'{(i + 1) % 10} - {ans}')

print(f'Ошибка в предсказании составила - {error}')

По итогу обучения модель считает:
1 - Простое
2 - Простое
3 - Простое
4 - Не простое
5 - Простое
6 - Не простое
7 - Простое
8 - Не простое
9 - Не простое
0 - Не простое
Ошибка в предсказании составила - 0.16101737223212786


### Задание 7.1.6.
**Задание:** Создать нейронную сеть со структурой «многослойный персептрон», и обучить ее распознаванию чисел,делящихся на 3 без остатка, заданных пиксельной матрицей размером 7х5. Произвести тестирование НС при добавлении шума.  
В переменную ***numbers_3_answer*** записываем данные на которых будет производится обучение модели, а также берем numbers из прошлого задания.

In [37]:
numbers_3_answer = np.array([[0,0,1,0,0,1,0,0,1,1]]).T

Создадим объект класса NeuralNetwork.

In [38]:
Numbers_3 = NeuralNetwork(dataset=numbers,
                          answer=numbers_3_answer,
                          neural_count=15,
                          iteration_count=10000,
                          func1=sigmoid,
                          func2=sigmoid)

Произведем обучение.

In [39]:
Numbers_3.Train()

На итерации 0 ошибка - 2.5691323938386124155
На итерации 1000 ошибка - 1.1073242389110018395
На итерации 2000 ошибка - 1.0486084699479524283
На итерации 3000 ошибка - 1.030759901410942031
На итерации 4000 ошибка - 1.0226900767681845961
На итерации 5000 ошибка - 1.0180781036672698534
На итерации 6000 ошибка - 1.0150743781378202239
На итерации 7000 ошибка - 1.0129503159196844171
На итерации 8000 ошибка - 1.011361505648661539
На итерации 9000 ошибка - 1.0101238291649589962
__________________________________________________
Итоговая ошибка 1.0091306310813316501

Обучение заняло 9.17531943321228 сек


Далее протестируем работу обученной модели, на модифицированных данных и посмотрим какой результат она выдаст.

In [40]:
print('На модифицированные данные нейронная сеть отвечает:')
error = 0

for i in range(len(numbers_mod)):
    data = numbers_mod[i:i+1]
    ans = Numbers_3.Predict(data)[0][0]
    error += np.sum((ans - Numbers_3.answer[i:i+1]) ** 2)
    ans = 'Делится на 3 без остатка' if round(ans) else 'Не делится на 3 без остатка'
    print(f'{(i + 1) % 10} - {ans}')

print(f'Ошибка в предсказании составила - {error}')

На модифицированные данные нейронная сеть отвечает:
1 - Не делится на 3 без остатка
2 - Не делится на 3 без остатка
3 - Делится на 3 без остатка
4 - Не делится на 3 без остатка
5 - Делится на 3 без остатка
6 - Делится на 3 без остатка
7 - Не делится на 3 без остатка
8 - Делится на 3 без остатка
9 - Не делится на 3 без остатка
0 - Делится на 3 без остатка
Ошибка в предсказании составила - 1.435167247185581


**Вывод**  
В данных задания мы проводили тесты на модифицированных данных, соответвено точность предсказания была ниже. Это обусловленно не большой выборкой данных. Так же не стоит забывать про возможное переобучение сети.

# Лабораторная работа 7.2.
## Искусственный нос

**Цель** - разработать и исслодовать ИНС обратного распределения для искусственного носа, предназначенного для химического анализа воздушной среды.  
В переменные ***impurity*** и ***impurity_answer*** записываем данные на которых будет производится обучение модели.

In [41]:
impurity = np.array([[1, 0.05, 0.1, 0.3, 0.07, 0.08, 0.2, 0.05, 0.2, 0.6, 0.8],
                     [0.8, 0.4, 0.7, 0.6, 0.1, 0.5, 1, 0.75, 0.5, 0.7, 0.8],
                     [0.9, 0.2, 0.4, 0.5, 0.1, 0.7, 0.6, 0.5, 0.5, 0.7, 0.8],
                     [0.85, 0.7, 0.8, 0.65, 0.1, 0.4, 1, 0.7, 0.4, 0.6, 0.7],
                     [0.9, 0.3, 0.3, 0.4, 0.04, 0.1, 0.5, 0.3, 0.2, 0.7, 0.8],
                     [0.95, 0.18, 0.21, 0.3, 0.05, 0.1, 0.3, 0.2, 0.2, 0.5, 0.7]], dtype=np.float128)



impurity_answer = np.array([[0, 0, 0, 0, 0, 1],  # Нет
                            [0, 0, 0, 0, 1, 0],  # Ацетон
                            [0, 0, 0, 1, 0, 0],  # Аммиак
                            [0, 0, 1, 0, 0, 0],  # Изопропанол
                            [0, 1, 0, 0, 0, 0],  # Белый "штрих"
                            [1, 0, 0, 0, 0, 0]]) # Уксус


Создадим объект класса NeuralNetwork. В данное задаче как и в задаче с определением буквы для классификации лучше всего использовать функцию **softmax**.

In [42]:
Impurity = NeuralNetwork(dataset=impurity,
                         answer=impurity_answer,
                         neural_count=15,
                         iteration_count=10000,
                         func2=softmax)
Impurity.GenerateWeights()

Произведем обучение.

In [43]:
Impurity.Train()

На итерации 0 ошибка - 8.246700422670458259
На итерации 1000 ошибка - 2.5970464507474085737e-05
На итерации 2000 ошибка - 4.7086903420172897942e-06
На итерации 3000 ошибка - 1.7900761639016368543e-06
На итерации 4000 ошибка - 9.1033416436891036594e-07
На итерации 5000 ошибка - 5.4138258031273930627e-07
На итерации 6000 ошибка - 3.550754661487755993e-07
На итерации 7000 ошибка - 2.4888190437295416145e-07
На итерации 8000 ошибка - 1.8308380681743554993e-07
На итерации 9000 ошибка - 1.3986387121062363635e-07
__________________________________________________
Итоговая ошибка 1.0997339600765824284e-07

Обучение заняло 3.4211952686309814 сек


Протестируем работу обученной сети.

In [44]:
print('По итогу обучения модель считает:')
for i in range(len(impurity)):
    data = impurity[i:i+1]
    ans = list(map(round, Impurity.Predict(data)[0]))
    print(f'{ans} - {data[0]}')

По итогу обучения модель считает:
[0, 0, 0, 0, 0, 1] - [1.   0.05 0.1  0.3  0.07 0.08 0.2  0.05 0.2  0.6  0.8 ]
[0, 0, 0, 0, 1, 0] - [0.8  0.4  0.7  0.6  0.1  0.5  1.   0.75 0.5  0.7  0.8 ]
[0, 0, 0, 1, 0, 0] - [0.9 0.2 0.4 0.5 0.1 0.7 0.6 0.5 0.5 0.7 0.8]
[0, 0, 1, 0, 0, 0] - [0.85 0.7  0.8  0.65 0.1  0.4  1.   0.7  0.4  0.6  0.7 ]
[0, 1, 0, 0, 0, 0] - [0.9  0.3  0.3  0.4  0.04 0.1  0.5  0.3  0.2  0.7  0.8 ]
[1, 0, 0, 0, 0, 0] - [0.95 0.18 0.21 0.3  0.05 0.1  0.3  0.2  0.2  0.5  0.7 ]


**Вывод**  
Так как сумма всех значений выходного слоя после использования **softmax** равна 1, несложно произвести классификацию при помощи округления значений. Итоговая точность значения будет 1.0997339600765824284e-07 соответсвенно.